In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import mt4_hst
%matplotlib inline
from stock_env.feature.feature_extractor import *
# pd.options.display.max_rows = 9999

In [8]:
ticker = 'PVD'
df = mt4_hst.read_hst("../stock_datasets/" + ticker + "1440.hst")
df.sort_values(by='time', inplace=True)
df.head()

,time,open,high,low,close,volume
0,2006-12-05,19.115231,19.115231,19.115231,19.115231,25500.0
1,2006-12-06,18.232988,18.232988,18.232988,18.232988,17860.0
2,2006-12-07,17.350748,17.350748,17.350748,17.350748,24160.0
3,2006-12-08,16.615545,16.615545,16.615545,16.615545,20800.0
4,2006-12-11,16.615545,17.350748,16.615545,17.350748,334090.0


In [ ]:
features = TrendFeatures().preprocess(df)
# cols = """
# time close ADX_20 AROOND_20 AROONU_20 AROONOSC_20 
# STC_10_10_20_0.5 STCstoch_10_10_20_0.5 
# NATR_20 RSI_20 CCI_20_0.015""".split()
# features[cols].head()

In [ ]:
features.head()

In [ ]:
help(ta.stc)

# Save DataLoader

In [5]:
from stock_env.data_loader import RandomStockLoader
from stock_env.feature.feature_extractor import TrendFeatures
# settings
data_path = "../stock_datasets/"

tickers = (
    "FPT MBB SSI VND HAX HAH VHC ANV HCM MBS "
    "VCI BCM IDC ACB HPG NKG TCB DCM DPM DGC " 
    "SZC HAG ROS FLC HBC VGC REE NT2 VSH POW "
).split()
    
feature_extractor = TrendFeatures

# data_loader = RandomStockLoader(
#     tickers = tickers,
#     data_folder_path = data_path,
#     feature_extractor = feature_extractor
# )

# save_path = "stock_env/datasets/30stocks"
# data_loader.save(save_path)

In [2]:
from stock_env.envs import *
import gym
env = gym.make("RandomVN30-v0")